In [1]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from prophet import Prophet
from datetime import datetime
from statsmodels.tsa.seasonal import STL
from statsmodels.tools.sm_exceptions import ValueWarning
from statsmodels.graphics.tsaplots import plot_acf
import warnings

warnings.simplefilter(action='ignore', category=ValueWarning)

c:\Users\jason\anaconda3\envs\playground\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
DATA_PATH = "https://raw.githubusercontent.com/Metaluv/Palette_Training/main/Palette_Training_5-main/data/rm-yields-data.csv"
GEO_PATH = "https://raw.githubusercontent.com/Metaluv/Palette_Training/main/Palette_Training_5-main/Rural_Municipality/Rural_Municipality.shp"

def load_data():
    return pd.read_csv(DATA_PATH)

In [13]:
def load_geo():
    return gpd.read_file(GEO_PATH)

In [14]:
def load_merged_data():
    geo_df = gpd.read_file(GEO_PATH)    
    geo_df.rename(columns={'RMNO': 'RM',}, inplace=True)
    geo_df['RM'] = geo_df['RM'].astype('int64')

    data_df = pd.read_csv(DATA_PATH)

    merged = geo_df.merge(data_df, on='RM', how='left')
    return merged

In [15]:
load_data()

,Year,RM,Winter Wheat,Canola,Spring Wheat,Mustard,Durum,Sunflowers,Oats,Lentils,Peas,Barley,Fall Rye,Canary Seed,Spring Rye,Tame Hay,Flax,Chickpeas
0,1938,1,NaN,NaN,4.00,NaN,NaN,NaN,1.00,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0,NaN
1,1939,1,NaN,NaN,9.00,NaN,NaN,NaN,16.00,NaN,NaN,16.0,NaN,NaN,NaN,NaN,0.0,NaN
2,1940,1,NaN,NaN,12.00,NaN,NaN,NaN,23.00,NaN,NaN,19.0,NaN,NaN,NaN,NaN,8.0,NaN
3,1941,1,NaN,NaN,18.00,NaN,NaN,NaN,32.00,NaN,NaN,28.0,NaN,NaN,NaN,NaN,5.0,NaN
4,1942,1,NaN,NaN,20.00,NaN,NaN,NaN,35.00,NaN,NaN,28.0,14.0,NaN,NaN,NaN,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25012,2017,622,NaN,43.06,59.49,NaN,NaN,NaN,93.01,NaN,43.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25013,2018,622,NaN,39.30,59.50,NaN,NaN,NaN,96.40,NaN,43.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN
25014,2019,622,NaN,46.10,63.90,NaN,NaN,NaN,106.00,NaN,33.7,73.9,NaN,NaN,NaN,NaN,NaN,NaN
25015,2020,622,NaN,40.90,55.10,NaN,NaN,NaN,94.50,NaN,49.2,74.3,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
load_geo()

2023-03-19 15:29:17.090 ERROR   fiona._env: `/vsimem/b0638bee8eed4c4eb3560596485b2fda' not recognized as a supported file format.


DriverError: '/vsimem/b0638bee8eed4c4eb3560596485b2fda' not recognized as a supported file format.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
import geopandas as gpd

In [19]:
# Check the version of pandas
print(pd.__version__)

1.5.3


In [20]:
# Read in geo data
geo_df = gpd.read_file('Rural_Municipality/Rural_Municipality.shp')

In [21]:
geo_df

,PPID,EFFDT,EXPDT,FEATURECD,RMNO,RMNM,SHAPE_AREA,SHAPE_LEN,geometry
0,0101000095,2019-01-21,NaN,RMPPID,095,GOLDEN WEST,8.101431e+08,2.658514e+05,"POLYGON ((654081.000 5546088.320, 654885.320 5..."
1,0101000378,2019-07-29,NaN,RMPPID,378,ROSEMOUNT,5.844701e+08,1.612719e+05,"POLYGON ((265258.740 5810148.180, 266062.740 5..."
2,0101000288,2015-01-27,NaN,RMPPID,288,PLEASANT VALLEY,8.532007e+08,1.168951e+05,"POLYGON ((254141.490 5701256.420, 254179.510 5..."
3,0101000106,2019-04-24,NaN,RMPPID,106,WHISKA CREEK,8.526283e+08,1.292883e+05,"POLYGON ((339874.810 5539057.770, 339849.430 5..."
4,0101000132,2019-07-16,NaN,RMPPID,132,HILLSBOROUGH,6.343913e+08,1.030527e+05,"POLYGON ((445175.620 5573313.600, 445572.880 5..."
...,...,...,...,...,...,...,...,...,...
293,0101000588,2022-12-19,NaN,RMPPID,588,MEADOW LAKE,5.922870e+09,1.011120e+06,"MULTIPOLYGON (((307104.520 6003527.820, 307139..."
294,0101000352,2022-12-21,NaN,RMPPID,352,HEART'S HILL,8.812504e+08,1.210734e+05,"POLYGON ((187054.680 5758478.840, 187005.860 5..."
295,0101000351,2022-12-21,NaN,RMPPID,351,PROGRESS,8.441731e+08,1.419927e+05,"POLYGON ((209380.850 5785495.100, 209851.950 5..."
296,0101000520,2022-12-21,NaN,RMPPID,520,PADDOCKWOOD,2.408683e+09,6.212794e+05,"MULTIPOLYGON (((451283.150 5981138.420, 451285..."
